In [ ]:
import json
from pathlib import Path

import duckdb
import plotly.express as px
import polars as pl
import s3fs

In [ ]:
with open(Path("creds/creds.json"), "r") as target:
    creds = json.load(target)
    storage_options = creds["AWS"]


In [ ]:
# pull gzip data from s3
s3 = s3fs.S3FileSystem(
    key=storage_options["aws_access_key_id"],
    secret=storage_options["aws_secret_access_key"],
)


filepath = f"s3://{storage_options['aws_bucket']}/datalog/{distrik}/{deviceid}/{hivehour}/{hivehour}"
bucketpath


In [ ]:
s3paths = s3.glob(filepath)
s3paths = [i.replace("smartdbucket/datalog/BRCG/SLS30I009/", "") for i in s3paths]

In [ ]:
# duckdb s3 connection config
conn = duckdb.connect()

conn.execute("INSTALL httpfs;")
conn.execute("LOAD httpfs;")

conn.execute(f"SET s3_region = '{storage_options['aws_region']}';")
conn.execute(f"SET s3_access_key_id = '{storage_options['aws_access_key_id']}';")
conn.execute(
    f"SET s3_secret_access_key = '{storage_options['aws_secret_access_key']}';"
)


In [ ]:
list_unit = [
    ("SLS30I009", "LD780"),
    ("SLS30I015", "LD781"),
    ("SLS30I016", "LD782"),
    ("SLS30I039", "LD776"),
    ("SLS30I146", "LD772"),
    ("SLS30I477", "LD778"),
    ("SDLIR038", "LD921"),
    ("SLS30I623", "LD783"),
]

In [ ]:
df = conn.sql(f"""
SELECT CAST(to_timestamp(heartbeat) as DATE)as tgl_utc,COUNT(1) as jml
FROM read_json_auto('{bucketpath}')
GROUP BY CAST(to_timestamp(heartbeat) as DATE)
ORDER BY CAST(to_timestamp(heartbeat) as DATE)
           """).pl()

df

In [ ]:
list_unit

In [ ]:
distrik = "BRCG"
deviceid = "SLS30I009"


def count_canbus_rows(devid, hivehour):
    bucketpath = f"s3://{storage_options['aws_bucket']}/datalog/{distrik}/{devid}/{hivehour}/{hivehour}.txt.gz"
    df = conn.sql(f"""
SELECT CAST(to_timestamp(heartbeat) as DATE)+ INTERVAL 8 HOUR as datetime_wita,gpsspeed,gpsnumsat,vehiclespeed,speedsource,
FROM read_json_auto('{bucketpath}')
WHERE gpsspeed > 0 AND VehicleSpeed > 0
           """).pl()
    print(devid, "unit has :", len(df), "rows")
    return df

In [ ]:
list_unit[1][1]

In [ ]:
hivehour = "2025121*"
for sls, unitno in list_unit:
    try:
        if main_df is not None:
            df = count_canbus_rows(sls, hivehour)
            main_df = pl.concat([main_df, df])
    except NameError:
        # Code to execute if 'df' variable has not been assigned
        main_df = count_canbus_rows(sls, hivehour)
        print("DataFrame variable is not defined.")

In [ ]:
# pull gzip data from s3
s3 = s3fs.S3FileSystem(
    key=storage_options["aws_access_key_id"],
    secret=storage_options["aws_secret_access_key"],
)


filepath = f"s3://{storage_options['aws_bucket']}/datalog/{distrik}/{deviceid}/{hivehour}/{hivehour}"
bucketpath


In [ ]:
bucketpath = f"s3://{storage_options['aws_bucket']}/datalog/{distrik}/{deviceid}/{hivehour}/{hivehour}.txt.gz"
df = conn.sql(f"""
SELECT CAST(to_timestamp(heartbeat) as DATE)+ INTERVAL 8 HOUR as datetime_wita,gpsspeed,gpsnumsat,vehiclespeed,speedsource,
FROM read_json_auto('{bucketpath}')
WHERE gpsspeed > 0 AND VehicleSpeed > 0 AND speedsource = 'CANBus

           """).pl()

In [ ]:
df.filter(pl.col("speedsource") == "CANBus")

In [ ]:
px.